In [ ]:
import numpy as np
import pandas as pd
import warnings
import joblib
import time
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge
import optuna

RANDOM_SEED = 0

warnings.filterwarnings("ignore")
optuna.logging.set_verbosity(optuna.logging.WARNING)
np.random.seed(RANDOM_SEED)

# =============================================================================
# CONFIGURATION
# =============================================================================
print("="*80)
print("RIDGE REGRESSION HYPERPARAMETER OPTIMIZATION")
print("="*80)

# =============================================================================
# LOAD PREPROCESSED DATA§
# =============================================================================
print("\n" + "="*80)
print("LOADING PREPROCESSED DATA")
print("="*80)

X_train = joblib.load('../../data/preprocessed/X_train.pkl')
y_train = joblib.load('../../data/preprocessed/y_train.pkl')
X_test = joblib.load('../../data/preprocessed/X_test.pkl')
metadata = joblib.load('../../data/preprocessed/metadata.pkl')

print(f"✓ Train shape: {X_train.shape}")
print(f"✓ Target shape: {y_train.shape}")
print(f"✓ Test shape: {X_test.shape}")

# =============================================================================
# OPTIMIZATION SETTINGS
# =============================================================================
FAST_MODE = False  # Set to False for more thorough search
N_TRIALS = 1 if FAST_MODE else 200
N_FOLDS = 5

print(f"\n{'='*60}")
print(f"OPTIMIZATION SETTINGS")
print(f"{'='*60}")
print(f"Mode: {'FAST' if FAST_MODE else 'THOROUGH'}")
print(f"Trials: {N_TRIALS}")
print(f"Folds: {N_FOLDS}")

# =============================================================================
# OBJECTIVE FUNCTION
# =============================================================================
def objective_ridge(trial, X, y):
    """Objective function for Ridge optimization"""
    
    # Hyperparameters to optimize
    params = {
        'alpha': trial.suggest_float('alpha', 1e-3, 1e3, log=True),
        'fit_intercept': trial.suggest_categorical('fit_intercept', [True, False]),
        'solver': trial.suggest_categorical('solver', ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']),
        'random_state': RANDOM_SEED,
    }

    start_time = time.time()

    print(f"\n{'='*60}")
    print(f"Trial {trial.number}")
    print(f"{'='*60}")
    print(f"Parametri testati: {params}")

    kf = KFold(n_splits=N_FOLDS, shuffle=True, random_state=RANDOM_SEED)
    scores = []

    for fold_idx, (train_idx, val_idx) in enumerate(kf.split(X)):
        X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
        
        # Handle both pandas Series and numpy arrays
        if isinstance(y, pd.Series):
            y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]
        else:
            y_tr, y_val = y[train_idx], y[val_idx]

        model = Ridge(**params)
        model.fit(X_tr, y_tr)

        preds = np.clip(model.predict(X_val), 0, 100)
        rmse = np.sqrt(mean_squared_error(y_val, preds))
        scores.append(rmse)

        print(f"  Fold {fold_idx + 1}: RMSE = {rmse:.4f}")
    
    optimization_time = time.time() - start_time
    mean_score = np.mean(scores)
    print(f"RMSE medio: {mean_score:.4f}")
    print(f"Elapsed Time: {optimization_time:.1f}s")
    print(f"{'='*60}\n")
    
    return mean_score

# =============================================================================
# RUN OPTIMIZATION
# =============================================================================
print("\n" + "="*80)
print("STARTING BAYESIAN OPTIMIZATION")
print("="*80)

start_time = time.time()

study = optuna.create_study(
    direction='minimize',
    sampler=optuna.samplers.TPESampler(seed=RANDOM_SEED)
)

study.optimize(
    lambda trial: objective_ridge(trial, X_train, y_train),
    n_trials=N_TRIALS,
    show_progress_bar=True,
    n_jobs=8
)

optimization_time = time.time() - start_time

# =============================================================================
# RESULTS
# =============================================================================
print("\n" + "="*80)
print("OPTIMIZATION RESULTS")
print("="*80)
print(f"Best RMSE: {study.best_value:.6f}")
print(f"Optimization time: {optimization_time:.1f}s")
print(f"\nBest parameters:")
for param, value in sorted(study.best_params.items()):
    print(f"  {param:20s}: {value}")

# =============================================================================
# SAVE RESULTS
# =============================================================================
# Save best parameters
best_params = study.best_params.copy()
best_params['random_state'] = RANDOM_SEED

joblib.dump(best_params, 'ridge_params.pkl')
print("\n✓ Parameters saved to: ridge_params.pkl")

# Save optimization history
history = pd.DataFrame({
    'trial': [t.number for t in study.trials],
    'value': [t.value for t in study.trials],
    'params': [str(t.params) for t in study.trials]
})
history.to_csv('ridge_history.csv', index=False)
print("✓ History saved to: ridge_history.csv")

# Save study object
joblib.dump(study, 'ridge_study.pkl')
print("✓ Study saved to: ridge_study.pkl")

# Save summary
summary = {
    'model': 'Ridge',
    'best_rmse': study.best_value,
    'n_trials': N_TRIALS,
    'n_folds': N_FOLDS,
    'optimization_time': optimization_time,
    'best_params': best_params
}
joblib.dump(summary, 'ridge_summary.pkl')
print("✓ Summary saved to: ridge_summary.pkl")

# =============================================================================
# TRAIN FINAL MODEL ON FULL TRAINING DATA
# =============================================================================
print("\n" + "="*80)
print("TRAINING FINAL MODEL ON FULL DATASET")
print("="*80)

final_model = Ridge(**best_params)

print("Training final model...")
final_model.fit(X_train, y_train)

# Save final model
joblib.dump(final_model, 'ridge_final_model.pkl')
print("\n✓ Final model saved to: ridge_final_model.pkl")

# =============================================================================
# GENERATE PREDICTIONS AND SUBMISSION FILE
# =============================================================================
print("\n" + "="*80)
print("GENERATING PREDICTIONS")
print("="*80)

# Make predictions on test set
test_predictions = final_model.predict(X_test)

# Clip predictions to valid range [0, 100]
test_predictions = np.clip(test_predictions, 0, 100)

# Create submission DataFrame
submission = pd.DataFrame({
    'id': range(len(X_train), len(test_predictions) + len(X_train)),
    'exam_score': test_predictions
})

# Save submission file
submission.to_csv('submission.csv', index=False)
print("✓ Submission file saved to: submission.csv")

print(f"\nSubmission statistics:")
print(f"  Min prediction: {test_predictions.min():.2f}")
print(f"  Max prediction: {test_predictions.max():.2f}")
print(f"  Mean prediction: {test_predictions.mean():.2f}")
print(f"  Median prediction: {np.median(test_predictions):.2f}")
print(f"  Std prediction: {test_predictions.std():.2f}")
print(f"  Shape: {submission.shape}")

# =============================================================================
# FEATURE IMPORTANCE (COEFFICIENTS)
# =============================================================================
print("\n" + "="*80)
print("FEATURE COEFFICIENTS")
print("="*80)

# Get feature coefficients
feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'coefficient': final_model.coef_
}).sort_values('coefficient', ascending=False, key=abs)

print("\nTop 10 features by absolute coefficient:")
print(feature_importance.head(10).to_string(index=False))

# Save feature importance
feature_importance.to_csv('ridge_coefficients.csv', index=False)
print("\n✓ Coefficients saved to: ridge_coefficients.csv")

# =============================================================================
# MODEL STATISTICS
# =============================================================================
print("\n" + "="*80)
print("MODEL STATISTICS")
print("="*80)
print(f"Intercept: {final_model.intercept_:.4f}")
print(f"Number of features: {len(final_model.coef_)}")
print(f"Alpha (regularization): {best_params['alpha']:.6f}")

# =============================================================================
# FINAL SUMMARY
# =============================================================================
print("\n" + "="*80)
print("RIDGE REGRESSION OPTIMIZATION COMPLETE!")
print("="*80)
print(f"✓ Best CV RMSE: {study.best_value:.6f}")
print(f"✓ Final model trained on {len(X_train)} samples")
print(f"✓ Predictions generated for {len(X_test)} test samples")
print(f"✓ All results saved")
print("\nFiles created:")
print("  • ridge_params.pkl")
print("  • ridge_history.csv")
print("  • ridge_study.pkl")
print("  • ridge_summary.pkl")
print("  • ridge_final_model.pkl")
print("  • ridge_coefficients.csv")
print("  • submission.csv")
print("="*80)

RIDGE REGRESSION HYPERPARAMETER OPTIMIZATION

LOADING PREPROCESSED DATA
✓ Train shape: (630000, 8)
✓ Target shape: (630000,)
✓ Test shape: (270000, 8)

OPTIMIZATION SETTINGS
Mode: THOROUGH
Trials: 200
Folds: 5

STARTING BAYESIAN OPTIMIZATION


  0%|          | 0/200 [00:00<?, ?it/s]


Trial 0
Parametri testati: {'alpha': 0.24030775161688736, 'fit_intercept': False, 'solver': 'svd', 'random_state': 0}

Trial 1
Parametri testati: {'alpha': 0.035352848022925834, 'fit_intercept': True, 'solver': 'cholesky', 'random_state': 0}

Trial 2
Parametri testati: {'alpha': 0.721352852544981, 'fit_intercept': False, 'solver': 'sparse_cg', 'random_state': 0}

Trial 3
Parametri testati: {'alpha': 0.0451985850311862, 'fit_intercept': True, 'solver': 'saga', 'random_state': 0}

Trial 4
Parametri testati: {'alpha': 1.6236777189136362, 'fit_intercept': False, 'solver': 'sparse_cg', 'random_state': 0}

Trial 5
Parametri testati: {'alpha': 0.007167873403227958, 'fit_intercept': True, 'solver': 'svd', 'random_state': 0}

Trial 6
Parametri testati: {'alpha': 9.75480169944639, 'fit_intercept': False, 'solver': 'svd', 'random_state': 0}

Trial 7
Parametri testati: {'alpha': 5.185714462607664, 'fit_intercept': True, 'solver': 'sag', 'random_state': 0}
  Fold 1: RMSE = 8.8903
  Fold 1: RMSE = 